In [60]:
import requests
from bs4 import BeautifulSoup
import datetime
import time
import os

base_url_pre_2023 = 'https://www.ag.ky.gov/Resources/orom/Pages/{}-OROM.aspx'
base_url_post_2023 = 'https://www.ag.ky.gov/Resources/orom/{}-OROM/Pages/default.aspx'
# Generate a list of years from 1993 to 2024
start_year = 1993
end_year = datetime.date.today().year

# Create a list of years
years = [year for year in range(start_year, end_year + 1)]

# Initialize an empty DataFrame to store all data
master_df = pd.DataFrame()

# Headers to mimic a request from my web browser
html_headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36',
    'Referer': 'https://www.ag.ky.gov/Resources/orom/Pages/default.aspx',
    'Sec-Ch-Ua': '"Google Chrome";v="123", "Not:A-Brand";v="8", "Chromium";v="123"'
}

# Strip html tags from a string.
def strip_html_bs(html_str):
    soup = BeautifulSoup(html_str, features="html.parser")
    return soup.get_text().strip()

# Get the href attribute form the first <a> tag found in a string.
def get_href(html_str):
    # Assuming the value is a string containing HTML
    soup = BeautifulSoup(html_str, 'html.parser')
    a_tag = soup.find('a')
    if a_tag:
        return a_tag.get('href')
    else:
        return None

# Take raw html from each cell and get the preprocessed values we want.
def process_row_data(row_dict):
    new_object = {}
    for key, value in row_dict.items():
        if key == "Download":
            new_object[key] = get_href(value)
        else:
            new_object[key] = strip_html_bs(value)
    return new_object

for year in years:
    # Fetch the page content with headers
    print('fetching ', year)
    url = base_url_pre_2023.format(year) if year < 2023 else base_url_post_2023.format(year)
    time.sleep(2)
    response = requests.get(url, headers=html_headers)
    
    # Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find the table (assuming it's the first table found)
    table = soup.find('table')
    
    # Extract table headers
    headers = [header.text for header in table.find_all('th')]
    
    # Extract table rows
    rows = table.find_all('tr')
    
    # Extract data from each row
    for row in rows:
        data = [cell.prettify() for cell in row.find_all('td')]
        # Create a dictionary with headers as keys and data as values
        row_dict = dict(zip(headers, data))
        row = process_row_data(row_dict)
        row["Year"] = str(year)
        # Convert the dictionary to a DataFrame and append it to the master DataFrame
        row_df = pd.DataFrame([row])
        master_df = pd.concat([master_df, row_df])

# Now master_df contains all the data from the tables across the pages
master_df.to_csv(os.path.join('opinions', 'kyag_scraped.csv'), index=False)

master_df.describe(include="all")

fetching  1993
fetching  1994
fetching  1995
fetching  1996
fetching  1997
fetching  1998
fetching  1999
fetching  2000
fetching  2001
fetching  2002
fetching  2003
fetching  2004
fetching  2005
fetching  2006
fetching  2007
fetching  2008
fetching  2009
fetching  2010
fetching  2011
fetching  2012
fetching  2013
fetching  2014
fetching  2015
fetching  2016
fetching  2017
fetching  2018
fetching  2019
fetching  2020
fetching  2021
fetching  2022
fetching  2023
fetching  2024


,Year,Title,Date,Parties,Number,Download,Year New
count,7512,7480,7480,7480,5789,7480,5789
unique,32,7447,3777,6181,282,7480,25
top,2023,,5/15/2003,Uriah Pasha/Kentucky State Reformatory,,https://www.ag.ky.gov/Resources/orom/1993/93ORD002.htm,2016
freq,373,3,22,34,200,1,280


In [26]:
pd.set_option('max_colwidth', None) # Show full width of columns
master_df.head()

,Year,Title,Date,Parties,Number,Download,Year New
0,1993,NaN,NaN,NaN,NaN,NaN,NaN
0,1993,93-ORD-002,1/19/1993,Richard W. Richards/City of West Buechel,2,https://www.ag.ky.gov/Resources/orom/1993/93ORD002.htm,1993
0,1993,93-ORD-004,1/19/1993,Richard W. Richards/City of West Buechel,4,https://www.ag.ky.gov/Resources/orom/1993/93ORD004.htm,1993
0,1993,93-ORD-005,1/25/1993,Nick Comer/City of Maysville,5,https://www.ag.ky.gov/Resources/orom/1993/93ORD005.htm,1993
0,1993,93-ORD-006,1/25/1993,Susan Gayle Reed/Eastern Kentucky University,6,https://www.ag.ky.gov/Resources/orom/1993/93ORD006.htm,1993


In [28]:
master_df.describe(include="all")

,Year,Title,Date,Parties,Number,Download,Year New
count,7512.000000,7480,7480,7480,5789,7480,5789
unique,NaN,7447,3777,6181,282,7480,25
top,NaN,,5/15/2003,Uriah Pasha/Kentucky State Reformatory,,https://www.ag.ky.gov/Resources/orom/1993/93ORD002.htm,2016
freq,NaN,3,22,34,200,1,280
mean,2009.178381,NaN,NaN,NaN,NaN,NaN,NaN
std,8.834029,NaN,NaN,NaN,NaN,NaN,NaN
min,1993.000000,NaN,NaN,NaN,NaN,NaN,NaN
25%,2002.000000,NaN,NaN,NaN,NaN,NaN,NaN
50%,2009.000000,NaN,NaN,NaN,NaN,NaN,NaN
75%,2017.000000,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
# Assuming master_df is your DataFrame and 'Parties' is the column of interest
rows_with_two_slashes = master_df[master_df['Parties'].str.count('/') >= 2]

# To get the count of such rows
count_of_rows_with_two_slashes = rows_with_two_slashes.shape[0]

print(rows_with_two_slashes)
print(f"Number of rows with two or more '/' characters in the 'Parties' column: {count_of_rows_with_two_slashes}")


    Year       Title        Date  \
0   1993  93-ORD-105   9/14/1993   
0   1994  94-ORD-003   1/11/1994   
0   1994  94-ORD-010   1/28/1994   
0   1994  94-ORD-075    6/7/1994   
0   1994  94-ORD-120  10/25/1994   
..   ...         ...         ...   
0   2024  24-ORD-075   3/20/2024   
0   2024  24-ORD-076   3/20/2024   
0   2024  24-ORD-077   3/20/2024   
0   2024  24-ORD-078   3/20/2024   
0   2024  24-ORD-079   3/20/2024   

                                                                                        Parties  \
0                                         Carol White/Office of the Lyon County Judge/Executive   
0                          Bill Estep/Office of Whitley County Judge/Executive, Jerry F. Taylor   
0                   Virginia Luttrell/Office of Pulaski County Judge/Executive Darrell Beshears   
0                                  Larry Dale Thacker/Office of the Pike County Judge/Executive   
0   Courier-Journal and Louisville Times Company/Office of the Jefferson

In [33]:
for row in rows_with_two_slashes['Parties'].to_list():
    print(row)

Carol White/Office of the Lyon County Judge/Executive
Bill Estep/Office of Whitley County Judge/Executive, Jerry F. Taylor
Virginia Luttrell/Office of Pulaski County Judge/Executive Darrell Beshears
Larry Dale Thacker/Office of the Pike County Judge/Executive
Courier-Journal and Louisville Times Company/Office of the Jefferson County Judge/Executive
Dave Baker/Franklin County Judge/Executive
R. Keith Cullinan/Louisville/Jefferson County Revenue Commission
Cheryl A. Martin/Office of Butler County Judge/Executive
Donald J. Ruberg/Kenton/Boone Counties Cable Television Board
Kimberly A. Hobbs/McCracken County Judge/Executive
Randy Skaggs/Office of the Leslie County Judge/Executive
Randy Skaggs/Office of the Marshall County Judge/Executive
Randy Skaggs/Office of the Bath County Judge/Executive
Randy Skaggs/Office of the Floyd County Judge/Executive
Randy Skaggs/Office of the Jackson County Judge/Executive
Randy Skaggs/Office of the Knott County Judge/Executive
Randy Skaggs/Office of the Kn

In [61]:
# It looks like when there are multiple, we can safely assume the appellant is before the first / and the agency is after the first /
master_df = master_df.dropna(subset=['Parties']) # Drops rows with NaN in 'Parties'

In [62]:
# Split the 'Parties' column at the first occurrence of '/'
master_df[['Appellant', 'Agency']] = master_df['Parties'].str.split('/', n=1, expand=True)

In [36]:
master_df.describe(include="all")

,Year,Title,Date,Parties,Number,Download,Year New,Appellant,Agency
count,7480.000000,7480,7480,7480,5789,7480,5789,7480,7470
unique,NaN,7447,3777,6181,282,7480,25,4133,2808
top,NaN,,5/15/2003,Uriah Pasha/Kentucky State Reformatory,,https://www.ag.ky.gov/Resources/orom/1993/93ORD002.htm,2016,Randy Skaggs,Kentucky State Police
freq,NaN,3,22,34,200,1,280,132,320
mean,2009.181283,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,8.832171,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1993.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,2002.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,2009.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,2017.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
master_df['Appellant'] = master_df['Appellant'].str.strip()
master_df['Agency'] = master_df['Agency'].str.strip()
master_df.describe(include="all")

,Year,Title,Date,Parties,Number,Download,Year New,Appellant,Agency
count,7480,7480,7480,7480,5789,7480,5789,7480,7470
unique,32,7447,3777,6181,282,7480,25,4052,2653
top,2023,,5/15/2003,Uriah Pasha/Kentucky State Reformatory,,https://www.ag.ky.gov/Resources/orom/1993/93ORD002.htm,2016,Randy Skaggs,Kentucky State Police
freq,372,3,22,34,200,1,280,156,331


In [64]:
# Remove 'In re: ' from the beginning of the 'Appellant' column
master_df['Appellant'] = master_df['Appellant'].str.replace('^In re: ', '', regex=True)
master_df.describe(include="all")

,Year,Title,Date,Parties,Number,Download,Year New,Appellant,Agency
count,7480,7480,7480,7480,5789,7480,5789,7480,7470
unique,32,7447,3777,6181,282,7480,25,4047,2653
top,2023,,5/15/2003,Uriah Pasha/Kentucky State Reformatory,,https://www.ag.ky.gov/Resources/orom/1993/93ORD002.htm,2016,Randy Skaggs,Kentucky State Police
freq,372,3,22,34,200,1,280,156,331


In [65]:
master_df.to_csv(os.path.join('opinions', 'kyag_scraped.csv'), index=False)

In [55]:
most_frequent_values = master_df['Appellant'].value_counts()

# To get the most frequent value(s)
most_frequent_value = most_frequent_values.idxmax()

# If you want to get the top N most frequent values, you can do:
N = 10 # Change this to the number of top values you want
top_N_values = most_frequent_values[:N].index.tolist()

print("Most frequent appellant:", most_frequent_value)
print("Top", N, "top appellants:", top_N_values)


Most frequent appellant: Randy Skaggs
Top 10 top appellants: ['Randy Skaggs', 'Uriah Pasha', 'Lawrence Trageser', 'The Courier-Journal', 'Chris Hawkins', 'Uriah M. Pasha', 'James Harrison', 'Lexington Herald-Leader', 'Leonel Martinez', 'The State Journal']


In [119]:
# AG site stores the OAGs separately with a different API and result structure
oag_base_search_url = "https://www.ag.ky.gov/_layouts/15/Fwk.Webparts.Agency.Ui/Search/Search.ashx?SiteUrl=https%3A%2F%2Fwww.ag.ky.gov%2FResources%2FOpinions&ListsToCount=%7C9328041e-55d9-4eee-a2f2-818633a986a6&ListId=9328041e-55d9-4eee-a2f2-818633a986a6&Lookup=431acfbd-683c-4cfb-8eed-ec00b33dbdf7%3A36%2C35%2C31%2C30%2C29%2C28%2C27%2C25%2C1%2C2%2C3%2C26%2C4%2C5%2C6%2C7%2C8%2C9%2C10%2C11%2C12%2C13%2C14%2C15%2C16%2C17%2C18%2C19%2C20%2C21%2C22%2C33%2C34%2C%7C&SearchText=&PageId={}&PageSize={}&ShowNoResultMessage=True&OperatorType=AND&SortField=&SortDirection=Ascending&SortValue=&Sort=Year+New%3ADescending%7CTitle%3ADescending%7C&ItemTemplateFields=URL%3BDate%3BTitle%3BSubject%3B&_=1711898189865"
next_page_id = 0
page_size = 1000
keep_going = True

html_headers = {
    'User-Agent': 'user-agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36',
    'Referer': 'https://www.ag.ky.gov/Resources/Opinions/Pages/Opinions.aspx',
}

oag_df = pd.DataFrame(columns=['Id', 'Url', 'FileSize', 'Title', 'Date', 'Year', 'Subject'])

# Define the dtypes dictionary
dtype_dict = {
    'Id': 'int', 
    'Url': 'object', 
    'FileSize': 'object',
    'Title': 'object',
    'Date': 'object',
    'Year': 'int',
    'Subject': 'object', 
}

# Convert each column to the specified dtype
for column, dtype in dtype_dict.items():
    oag_df[column] = oag_df[column].astype(dtype)


while keep_going:
    # Fetch the page content with headers
    print('\nfetching page_id:', next_page_id, '\ncurrent total:', oag_df.shape[0])
    url = oag_base_search_url.format(next_page_id, page_size)
    time.sleep(2)
    response = requests.get(url, headers=html_headers)
    
    if data['HasNext']:
        next_page_id = data['PageNextId']
    else:
        keep_going = False
    
    data = response.json()
    
    # Extract data from each row
    for item in data['Items']:
        """
        Ex: item
        {
            "Id":721,
            "Url":"https://www.ag.ky.gov/Resources/Opinions/Opinions/OAG94-064.htm",
            "Fields":[
                {"Name":"Url","Value":"https://www.ag.ky.gov/Resources/Opinions/Opinions/OAG94-064.htm"},
                {"Name":"FileSize","Value":"4.88 KB"},{"Name":"Title","Value":"OAG 94-064"},
                {"Name":"Title","Value":"OAG 94-064"},
                {"Name":"Date","Value":"11/1/1994"},
                {"Name":"Subject","Value":"Whether a county attorney may supplement a sum set for his or her salary by the fiscal court, with monies obtained in connection with collection of delinquent taxes or related actions"}
            ]
        }
        """
        # Initialize an empty dictionary for the flattened object
        flattened_object = {"Id": item["Id"], "Url": item["Url"]}

        # Iterate over the "Fields" list
        for field in item["Fields"]:
            # Add each "Name" and "Value" to the flattened object
            flattened_object[field["Name"]] = field["Value"]
            
        # Get year off of the date
        flattened_object['Year'] = int(flattened_object['Date'].split('/')[-1])
        row_df = pd.DataFrame([flattened_object])
        oag_df = pd.concat([oag_df, row_df])

# Now master_df contains all the data from the tables across the pages
oag_df.to_csv(os.path.join('opinions', 'kyag_oag_scraped.csv'), index=False)

oag_df.describe(include="all")


fetching page_id: 0 
current total: 0

fetching page_id: 623 
current total: 663


,Id,Url,FileSize,Title,Date,Year,Subject
count,975.000000,975,975,975,975,975.000000,975
unique,NaN,663,452,663,507,NaN,661
top,NaN,https://www.ag.ky.gov/Resources/Opinions/Opinions/Opinion%20of%20the%20Attorney%20General%20%2024-1.pdf,35.00 KB,OAG 24-1,2/25/1992,NaN,Relates to the accuracy of computations to be used in adjusting salaries of constitutional officers in relation to changes in the Consumer Price Index.
freq,NaN,2,14,2,10,NaN,4
mean,690.742564,NaN,NaN,NaN,NaN,2002.726154,NaN
std,352.215223,NaN,NaN,NaN,NaN,11.343339,NaN
min,1.000000,NaN,NaN,NaN,NaN,1992.000000,NaN
25%,408.000000,NaN,NaN,NaN,NaN,1993.000000,NaN
50%,761.000000,NaN,NaN,NaN,NaN,1996.000000,NaN
75%,1006.000000,NaN,NaN,NaN,NaN,2016.000000,NaN


In [120]:
unique_years = oag_df['Year'].unique()
print(unique_years)


[2024 2023 2022 2021 2020 2019 2018 2017 2016 2015 2014 2013 2012 2011
 2010 2009 2008 2007 2006 2005 2004 2003 2002 2001 2000 1999 1998 1997
 1996 1995 1994 1993 1992]


In [133]:
import pandas as pd
import os
import requests

# Sort the DataFrame by "Date" in descending order
oag_df['Date'] = oag_df['Date'].apply(pd.to_datetime)
oag_df = oag_df.sort_values(by="Date", ascending=False)

# Define the directory where you want to save the files
directory = "opinions"

# Iterate through each row of the DataFrame
for index, row in oag_df.iterrows():
    # Construct the target directory path
    target_dir = os.path.join(directory, str(row["Year"]), row["Title"])
    
    # Create the target directory if it doesn't exist
    os.makedirs(target_dir, exist_ok=True)
    
    # Construct the target file path
    # Assuming the file name is the last part of the URL in the "Download" column
    file_name = row["Url"].split('/')[-1]
    target_file = os.path.join(target_dir, file_name)
    
    # Check if the file already exists
    if not os.path.exists(target_file):
        # Download the file
        response = requests.get(row["Url"], headers=html_headers)
        
        if response.status_code == 200:
            # Save the file to the target directory
            with open(target_file, 'wb') as f:
                f.write(response.content)
            print(f"{file_name} downloaded successfully.")
        else:
            print(f"Failed to download {row['Url']}. HTTP response status code: {response.status_code}")
    


Opinion%20of%20the%20Attorney%20General%20%2024-1.pdf downloaded successfully.
Opinion%20of%20the%20Attorney%20General%2023-10.pdf downloaded successfully.
Opinion%20of%20the%20Attorney%20General%2023-11.pdf downloaded successfully.
Opinion%20of%20the%20Attorney%20General%2023-09.pdf downloaded successfully.
Opinion%20of%20the%20Attorney%20General%2023-08.pdf downloaded successfully.
Opinion%20of%20the%20Attorney%20General%2023-07.pdf downloaded successfully.
Opinion%20of%20the%20Attorney%20General%2023-06.pdf downloaded successfully.
Opinion%20of%20the%20Attorney%20General%2023-05.pdf downloaded successfully.
Opinion%20of%20the%20Attorney%20General%2023-04.pdf downloaded successfully.
Opinion%20of%20the%20Attorney%20General%2023-03.pdf downloaded successfully.
Opinion%20of%20the%20Attorney%20General%2023-02.pdf downloaded successfully.
Opinion%20of%20the%20Attorney%20General%2023-01.pdf downloaded successfully.
OAG%2022-06.pdf downloaded successfully.
OAG%2022-05.pdf downloaded succes

In [135]:
import sys

def print_statusline(msg: str):
    last_msg_length = len(getattr(print_statusline, 'last_msg', ''))
    print(' ' * last_msg_length, end='\r')
    print(msg, end='\r')
    sys.stdout.flush() # Ensures the output is immediately visible
    setattr(print_statusline, 'last_msg', msg)

In [140]:
import pandas as pd
import os
import requests
import json

# Sort the DataFrame by "Date" in descending order
master_df['Date'] = master_df['Date'].apply(pd.to_datetime)
master_df = master_df.sort_values(by="Date", ascending=False)

# Define the directory where you want to save the files
directory = "opinions"

# Iterate through each row of the DataFrame
for index, row in master_df.iterrows():
    if isinstance(row["Download"], str) == False:
        continue;
    
    # Construct the target directory path
    target_dir = os.path.join(directory, str(row["Year"]), row["Title"])
    
    # Create the target directory if it doesn't exist
    os.makedirs(target_dir, exist_ok=True)
    
    # Construct the target file path
    # Assuming the file name is the last part of the URL in the "Download" column
    file_name = row["Download"].split('/')[-1]
    target_file = os.path.join(target_dir, file_name)
    
    # Check if the file already exists
    if not os.path.exists(target_file):
        # Download the file
        response = requests.get(row["Download"], headers=html_headers)
        
        if response.status_code == 200:
            # Save the file to the target directory
            with open(target_file, 'wb') as f:
                f.write(response.content)
            print_statusline(f"{file_name} downloaded successfully.")
        else:
            print(f"Failed to download {row['Download']}. HTTP response status code: {response.status_code}")
    
